# Julia 深度學習：卷積神經網路模型簡介

## 作業 033：訓練 CNN 學習門牌號碼資料集

訓練一個 CNN 模型來學習門牌號碼資料集。

In [4]:
using Flux
using Flux.Data: DataLoader
using Flux: @epochs, onecold, onehotbatch, throttle, logitcrossentropy
using MLDatasets
using Statistics

## 讀取資料

In [5]:
train_X, train_y = SVHN2.traindata(Float32, 1:20000)
test_X,  test_y  = SVHN2.testdata(Float32, 1:2000)

(Float32[0.14901961 0.15294118 … 0.19607843 0.1882353; 0.15294118 0.15294118 … 0.2 0.1882353; … ; 0.16470589 0.16862746 … 0.1764706 0.17254902; 0.15294118 0.15294118 … 0.16470589 0.16470589]

Float32[0.40392157 0.40784314 … 0.45882353 0.4509804; 0.40784314 0.40784314 … 0.4627451 0.4509804; … ; 0.40392157 0.39607844 … 0.45490196 0.4509804; 0.38039216 0.38039216 … 0.44313726 0.44313726]

Float32[0.23529412 0.23921569 … 0.29803923 0.2901961; 0.23921569 0.23921569 … 0.3019608 0.2901961; … ; 0.24313726 0.24705882 … 0.28235295 0.2784314; 0.22352941 0.22352941 … 0.27058825 0.2784314]

Float32[0.5058824 0.5254902 … 0.5411765 0.5137255; 0.49803922 0.52156866 … 0.50980395 0.47843137; … ; 0.48235294 0.49411765 … 0.39607844 0.43529412; 0.48235294 0.49019608 … 0.4392157 0.48235294]

Float32[0.5568628 0.5882353 … 0.59607846 0.5686275; 0.56078434 0.58431375 … 0.5647059 0.53333336; … ; 0.5254902 0.5372549 … 0.41960785 0.4627451; 0.5294118 0.5372549 … 0.4627451 0.50980395]

Float32[0.6 0.627451 … 0.647

In [6]:
train_y = onehotbatch(train_y, 1:10)
test_y = onehotbatch(test_y, 1:10)

10×2000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 0  0  1  0  0  1  0  1  1  0  0  0  0  …  0  0  0  0  0  0  1  0  0  0  1  0
 0  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  1  0  0  0  0  0  1  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  1     0  0  0  0  1  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  1  0  …  0  0  0  1  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  1  0  0  0  0  0  1  0  1  0  0
 0  0  0  0  0  0  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0

In [7]:
batchsize = 1024
train = DataLoader(train_X, train_y, batchsize = batchsize, shuffle = true)
test = DataLoader(test_X, test_y, batchsize = batchsize)

DataLoader((Float32[0.14901961 0.15294118 … 0.19607843 0.1882353; 0.15294118 0.15294118 … 0.2 0.1882353; … ; 0.16470589 0.16862746 … 0.1764706 0.17254902; 0.15294118 0.15294118 … 0.16470589 0.16470589]

Float32[0.40392157 0.40784314 … 0.45882353 0.4509804; 0.40784314 0.40784314 … 0.4627451 0.4509804; … ; 0.40392157 0.39607844 … 0.45490196 0.4509804; 0.38039216 0.38039216 … 0.44313726 0.44313726]

Float32[0.23529412 0.23921569 … 0.29803923 0.2901961; 0.23921569 0.23921569 … 0.3019608 0.2901961; … ; 0.24313726 0.24705882 … 0.28235295 0.2784314; 0.22352941 0.22352941 … 0.27058825 0.2784314]

Float32[0.5058824 0.5254902 … 0.5411765 0.5137255; 0.49803922 0.52156866 … 0.50980395 0.47843137; … ; 0.48235294 0.49411765 … 0.39607844 0.43529412; 0.48235294 0.49019608 … 0.4392157 0.48235294]

Float32[0.5568628 0.5882353 … 0.59607846 0.5686275; 0.56078434 0.58431375 … 0.5647059 0.53333336; … ; 0.5254902 0.5372549 … 0.41960785 0.4627451; 0.5294118 0.5372549 … 0.4627451 0.50980395]

Float32[0.6 0.627

## CNN 模型

In [8]:
model = Chain(
    Conv((3, 3), 3 => 32, pad = (1, 1), relu),
    MaxPool((2,2)),
    Conv((3, 3), 32 => 32, pad = (1, 1), relu),
    MaxPool((2,2)),
    Conv((3, 3), 32 => 64, pad = (1, 1), relu),
    MaxPool((2, 2)),
    flatten,
    Dense(1024, 256, relu),
    Dense(256, 10),
    softmax)

Chain(Conv((3, 3), 3=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 32=>32, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 32=>64, relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), flatten, Dense(1024, 256, relu), Dense(256, 10), softmax)

## 損失函數

In [9]:
loss(x, y) = logitcrossentropy(model(x), y)

loss (generic function with 1 method)

## Callback 函式

In [10]:
function test_loss()
    l = 0f0
    for (x, y) in test
        l += loss(x, y)
    end
    l / length(test)
end

test_loss (generic function with 1 method)

In [12]:
evalcb() = @show(test_loss())

evalcb (generic function with 1 method)

## 模型訓練

In [18]:
epochs = 50
@epochs epochs Flux.train!(loss, params(model), train, ADAM(0.005), cb = throttle(evalcb, 10))

┌ Info: Epoch 1
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 1.99487f0
test_loss() = 2.002821f0
test_loss() = 1.9850912f0
test_loss() = 1.9863346f0


┌ Info: Epoch 2
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.0073686f0
test_loss() = 1.984f0
test_loss() = 1.9813449f0
test_loss() = 1.9766138f0


┌ Info: Epoch 3
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


...

test_loss() = 1.7569004f0
test_loss() = 1.747351f0
test_loss() = 1.7560059f0
test_loss() = 1.7524844f0
test_loss() = 1.7523167f0


┌ Info: Epoch 49
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 1.7609181f0
test_loss() = 1.7500011f0
test_loss() = 1.7475426f0
test_loss() = 1.7449424f0
test_loss() = 1.7477477f0


┌ Info: Epoch 50
└ @ Main C:\Users\user\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 1.7523129f0
test_loss() = 1.7463562f0
test_loss() = 1.7557039f0
test_loss() = 1.7492623f0
test_loss() = 1.7473626f0


## 模型評估

In [19]:
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))

accuracy (generic function with 1 method)

In [20]:
accuracy(test_X, test_y)

0.713